In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# Generate synthetic data
def generate_data(a=2, c=3, num_samples=100):
    X = np.linspace(-10, 10, num_samples)
    Z = a * X + c + np.random.normal(0, 1, num_samples)
    return X, Z

X, Z = generate_data()
X_train, Z_train = X[:80], Z[:80]  # 80% for training
X_test, Z_test = X[80:], Z[80:]  # 20% for testing

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1,))
])

model.compile(optimizer='sgd', loss='mean_squared_error')

# Train the model
model.fit(X_train, Z_train, epochs=100, validation_split=0.2)

# Convert Keras model to a concrete function
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]

# Save frozen graph from frozen ConcreteFunction to .pb file
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="linear_regression_model.pb",
                  as_text=False)


Epoch 1/100
2/2 [==============================] - 0s 154ms/step - loss: 54.5724 - val_loss: 44.3265
Epoch 2/100
2/2 [==============================] - 0s 11ms/step - loss: 6.3014 - val_loss: 30.2253
Epoch 3/100
2/2 [==============================] - 0s 15ms/step - loss: 4.7722 - val_loss: 27.4758
Epoch 4/100
2/2 [==============================] - 0s 12ms/step - loss: 4.5827 - val_loss: 26.1113
Epoch 5/100
2/2 [==============================] - 0s 12ms/step - loss: 4.4824 - val_loss: 24.5344
Epoch 6/100
2/2 [==============================] - 0s 11ms/step - loss: 4.3599 - val_loss: 23.4501
Epoch 7/100
2/2 [==============================] - 0s 12ms/step - loss: 4.1649 - val_loss: 23.2851
Epoch 8/100
2/2 [==============================] - 0s 13ms/step - loss: 4.0806 - val_loss: 21.9764
Epoch 9/100
2/2 [==============================] - 0s 17ms/step - loss: 3.9063 - val_loss: 21.9008
Epoch 10/100
2/2 [==============================] - 0s 13ms/step - loss: 3.7675 - val_loss: 21.4678
Epoch 1

2024-02-11 07:32:06.565234: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-02-11 07:32:06.565550: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


'./frozen_models/linear_regression_model.pb'